In [ ]:
import sqlite3
import copy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import contextily as cx

In [ ]:
# GLOBAL VARIABLES/CONSTANTS

# SQLite File Path formatter
__SQLITE_0 = "<PATH_TO_DATABASE_1>.sqlite"
__SQLITE_1 = "<PATH_TO_DATABASE_2>.sqlite"
__SQLITE_2 = "<PATH_TO_DATABASE_3>.sqlite"


# sections.shp File Path
__SECTION_SHP = "Data/sections.shp"

# Number of experiments
__NUM_EXP = 11

# Output File Directory
__OUTPUT = "output/"

# SQL Query to be excecuted for different tables
__SQL_EXTRACT_MISECT_QUERY = 'SELECT * FROM MISECT'
__SQL_EXTRACT_MILANE_QUERY = 'SELECT * FROM MILANE'
__SQL_EXTRACT_MISYS_QUERY = 'SELECT * FROM MISYS'

# Columns to extract from different tables
__MISECT_COLUMNS = ['ent', 'eid', 'sid', 'flow_capacity', 'speed', 'travel', 'traveltime', 'density', 'flow', 'dtime']
__MILANE_COLUMNS = ['ent', 'eid', 'sid', 'lane', 'flow', 'speed', 'density', 'input_flow']
__MISYS_COLUMNS = ['ent', 'sid', 'density', 'speed', 'travel', 'traveltime', 'flow', 'dtime', 'qmean']

# Actual time for each time step
__TIME_REAL = ['14:15', '14:30', '14:45', '15:00', '15:15', '15:30', '15:45', '16:00', '16:15', '16:30', '16:45', '17:00', '17:15', '17:30', '17:45', '18:00', '18:15', '18:30', '18:45', '19:00', '19:15', '19:30', '19:45', '20:00']


# Scenario Names
__SCENARIOS = [
    "0% Dynamic En-Route",
    "10% Dynamic En-Route",
    "20% Dynamic En-Route",
    "30% Dynamic En-Route",
    "40% Dynamic En-Route",
    "50% Dynamic En-Route",
    "60% Dynamic En-Route",
    "70% Dynamic En-Route",
    "80% Dynamic En-Route",
    "90% Dynamic En-Route",
    "100% Dynamic En-Route"
]
__SCENARIO_PRECENTAGES = [
    "0%",
    "10%",
    "20%",
    "30%",
    "40%",
    "50%",
    "60%",
    "70%",
    "80%",
    "90%",
    "100%",
]

__XTICKS_LABELS_AVG = [
    "Mean Flow (veh/h)",
    "Mean Density (veh/km/lane)",
    "Mean Speed (km/h)",
    "Mean Travel Time (sec/km)",
    "Mean Delay Time (sec)",
    "Mean Queue (veh)"
]
__XTICKS_LABELS_AVG_SHORT = [
    "Flow \n(veh/h)",
    "Density \n(veh/km/lane)",
    "Speed \n(km/h)",
    "Travel Time \n(sec/km)",
    "Delay Time \n(sec)",
    "Queue \n(veh)"
]

In [ ]:
# Create a SQL connection to our SQLite database

# A list of established connections to our databases
con_0 = []
con_1 = []
con_2 = []

for i in range(__NUM_EXP):
    con_0.append(sqlite3.connect(__SQLITE_0.format(number=i)))
    con_1.append(sqlite3.connect(__SQLITE_1.format(number=i)))
    con_2.append(sqlite3.connect(__SQLITE_2.format(number=i)))

# Run SQL query and convert SQL to DataFrame

# List of dataframes extracted from each experiment
df_0_misys = []
df_1_misys = []
df_2_misys = []
for i in range(__NUM_EXP):
    # Run SQL
    query_0 = pd.read_sql(__SQL_EXTRACT_MISYS_QUERY, con_0[i])
    query_1 = pd.read_sql(__SQL_EXTRACT_MISYS_QUERY, con_1[i])
    query_2 = pd.read_sql(__SQL_EXTRACT_MISYS_QUERY, con_2[i])
    
    # Convert SQL to DataFrame
    df_0 = pd.DataFrame(query_0, columns = __MISYS_COLUMNS)
    df_0_misys.append(df_0)
    df_1 = pd.DataFrame(query_1, columns = __MISYS_COLUMNS)
    df_1_misys.append(df_1)
    df_2 = pd.DataFrame(query_2, columns = __MISYS_COLUMNS)
    df_2_misys.append(df_2)


In [ ]:
# Read the sections.shp shapefile
sections = gpd.read_file(__SECTION_SHP)
sections.crs

In [ ]:
sections

In [ ]:
sections_rep = sections[sections['eid'] == str(243)]

In [ ]:
ax = sections_rep.plot(figsize=(15, 15))
cx.add_basemap(ax, crs='EPSG:32610', source=cx.providers.CartoDB.Voyager) #4326
plt.title('Road Sections')

In [ ]:
sections_temp = sections.pivot_table(index=['eid'], aggfunc='size')
sections_temp[sections_temp > 1]

In [ ]:
def compute_average_results(df_list: list) -> pd.DataFrame:
    """
    Given a list of pd.DataFrame, compute the average of the specified columns

    Args:
    -----
    df_list: list
        A list of pd.DataFrame

    Returns:
    --------
    processed_df: pd.DataFrame
        A pd.DataFrame containing the average of the specified columns
    """
    processed_dfs = []
    for df in df_list:
        # Compute the mean of the specified columns
        mean_df = df[['flow', 'density', 'speed', 'traveltime', 'dtime', 'qmean']].mean()
        processed_dfs.append(mean_df)
    # Concatenate the series into a DataFrame
    processed_df = pd.concat(processed_dfs, axis=1).T
    processed_df.insert(0, "Scenario", __SCENARIOS)
    # processed_df.set_index("Scenario", inplace=True)
    return processed_df

In [ ]:
df_0_misys_avg = compute_average_results(df_0_misys)
df_0_misys_avg

In [ ]:
df_1_misys_avg = compute_average_results(df_1_misys)
df_1_misys_avg

In [ ]:
df_2_misys_avg = compute_average_results(df_2_misys)
df_2_misys_avg

In [ ]:
# Drop the "Scenario" column from each DataFrame
df_0_misys_avg_numeric = df_0_misys_avg.drop("Scenario", axis=1)
df_1_misys_avg_numeric = df_1_misys_avg.drop("Scenario", axis=1)
df_2_misys_avg_numeric = df_2_misys_avg.drop("Scenario", axis=1)

# Concatenate the DataFrames along the column axis
concatenated_df_misys_all = pd.concat([df_0_misys_avg_numeric, df_1_misys_avg_numeric, df_2_misys_avg_numeric], axis=1)
# concatenated_df_misys_all = pd.concat([df_0_misys_avg, df_1_misys_avg, df_2_misys_avg], axis=1)

# Compute the mean along the row axis
mean_df = concatenated_df_misys_all.groupby(concatenated_df_misys_all.columns, axis=1).mean()
mean_df = mean_df.round(4)
mean_df = mean_df[['flow', 'density', 'speed', 'traveltime', 'dtime', 'qmean']]

# Add the "Scenario" column back
mean_df.insert(0, "Scenario", __SCENARIOS)
# mean_df.set_index("Scenario", inplace=True)
mean_df

In [ ]:
mean_df.to_csv(__OUTPUT + "misys_avg_all.csv")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Define a dictionary for the "better" rule
better_rule = {'flow': 'higher', 'speed': 'higher', 'density': 'lower', 'traveltime': 'lower', 'dtime': 'lower', 'qmean': 'lower'}

# Standardize the columns according to the "better" rule
mean_df_standardized = mean_df.copy()
for col in better_rule.keys():
    if better_rule[col] == 'higher':
        # For "higher is better", we standardize by subtracting the min and dividing by the range
        mean_df_standardized[col] = (mean_df[col] - mean_df[col].min()) / (mean_df[col].max() - mean_df[col].min())
    else:
        # For "lower is better", we standardize by subtracting the value from max and dividing by the range
        mean_df_standardized[col] = (mean_df[col].max() - mean_df[col]) / (mean_df[col].max() - mean_df[col].min())

# Drop the "Scenario" column for the heatmap
mean_df_standardized_numeric = mean_df_standardized.drop("Scenario", axis=1)

# Create the heatmap
plt.figure(figsize=(12, 10))
sns.set(font_scale=1.5)
htmp = sns.heatmap(
    mean_df_standardized_numeric, 
    annot=mean_df.drop("Scenario", axis=1).values, 
    xticklabels=__XTICKS_LABELS_AVG_SHORT, 
    yticklabels=__SCENARIO_PRECENTAGES, fmt='.2f', 
    cmap='RdYlGn', 
    linewidths=0.5,
    
)
htmp.set(ylabel="Dynamic En-Route")
plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.title('Heatmap of Average Metrics for All Scenarios')
plt.savefig(__OUTPUT + "misys_avg_all_heatmap_short.png", dpi=300, bbox_inches='tight')
plt.show()
